# Writing a Newsletter with LLMs

> Summarizing insights articles for newsletter entries

In [86]:
from langchain import PromptTemplate
from langchain.schema.document import Document
from langchain_community.llms import Ollama
from langchain.chains.summarize import load_summarize_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader, PyPDFLoader

In [2]:
llm = Ollama(model="llama3.1")

In [4]:
# test the llm
res = llm.invoke("Tell me a joke")
print(res)

Here's one:

What do you call a fake noodle?

An impasta.

I hope that made you laugh! Do you want to hear another one?


In [ ]:
"C:\Users\e0h0afh\Desktop\3P Vendors\newsletter_resources\global-retail-outlook-2024.pdf"

In [61]:
# load a single document
def load_documents(path: str):
    """
    Loads a Single PDF document
    """
    document_loader = PyPDFLoader(path)
    return document_loader.load()

DELOITTE_PATH = "../newsletter_resources/global-retail-outlook-2024.pdf"
deloitte_document = load_documents(DELOITTE_PATH)

In [62]:
# cut out beginning and end which don't include useful information
deloitte_document = deloitte_document[2:21]

# combine all pages as document loader automatically splits into pages
combined_deloitte = ""
for page in deloitte_document:
    combined_deloitte += page.page_content

In [73]:
# number of words
len(combined_deloitte.split())

9784

In [76]:
def split_documents(documents: list[Document]):
    """
    Split our documents as they are too long each
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=10000,
        chunk_overlap=1000,
        #length_function=len,
        #is_separator_regex=False,
    )
    return text_splitter.create_documents(documents)

split_del = split_documents([combined_deloitte])
print(len(split_del))

8


In [80]:
# see the number of tokens
llm.get_num_tokens(split_del[2].page_content)

2563

## Create Chain

In [81]:
# run summarizer
chain = load_summarize_chain(llm, 
                             chain_type="map_reduce")

C:\Users\e0h0afh\AppData\Local\anaconda3\envs\news_rags\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
# Run chain with default prompts
output_summary = chain.run(split_del)

In [82]:
print(output_summary)

Here is a concise summary:

The global economy faces uncertainty with challenges in the US, European, Chinese, and Australian economies. The retail industry also faces headwinds, including inflation, cost pressures, and declining consumer spending. Key priorities for retailers include addressing climate change, improving ESG practices, and complying with regulations.

Retailers are expected to invest in new technology and business models, such as digital twins, automated micro-fulfillment centers, and cashierless checkout technology. The use of AI/GenAI is also on the rise, with top priorities including enhancing supply chain visibility, powering customer service chatbots, and improving predictive analytics.

The report highlights opportunities for retailers in health and wellness and technology-driven shifts from mass to micro retailing. It also emphasizes the importance of sustainability and adaptability in the face of changing consumer behaviors and technological advancements.


In [85]:
# See the default prompts used in the chain
print(chain.llm_chain.prompt.template)
print(chain.combine_document_chain.llm_chain.prompt.template)

Write a concise summary of the following:


"{text}"


CONCISE SUMMARY:
Write a concise summary of the following:


"{text}"


CONCISE SUMMARY:


### Setting Our Own Prompts

In [161]:
map_prompt = """
Write a brief summary of the main points in the business article snippet delimited by triple backquotes below.
Make sure to include relevant supporting information and data.
Do NOT use bulletpoints.

```{text}```

SUMMARY:
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

In [166]:
combine_prompt = """
Summarize the following text into an informative 500 word newsletter article. Use supporting data where possible, avoid bulletpoints.

```{text}```

NEWSLETTER ENTRY:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

In [167]:
# run summarizer
chain = load_summarize_chain(llm, 
                             chain_type="map_reduce",
                             map_prompt = map_prompt_template,
                             combine_prompt = combine_prompt_template,
                             verbose=True
                            )

In [168]:
# Run chain with custom prompts
output_summary = chain.run(split_del)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a brief summary of the main points in the business article snippet delimited by triple backquotes below.
Make sure to include relevant supporting information and data.
Do NOT use bulletpoints.

```Deloitte Global Retail Outlook 2024 | Navigating challenges and embracing opportunities - Insights from retail leaders around the world  03After the disruption that came about during by the COVID-19 
pandemic,	the	rising	costs-of-living,	inflationary	pressures	and 	
geopolitical tensions around the world, you could forgive retailers 
for being cautious about prospects for the year ahead. But the 
opposite appears to be true. 
Buoyed by opportunities founded in technology – not least of 
which, the emergence of generative AI – which could reduce costs, 
improve productivity and enhance the customer experience, the 
retailers interviewed as part of the Deloitte Global Retail Outl

In [169]:
print(output_summary)

**Global Economy to See Modest Growth in 2023 and Beyond**

The global economy is expected to experience modest growth in the coming years, despite facing various headwinds. According to recent forecasts, the US economy will likely see moderate growth driven by consumer spending and investment, while Europe's economic growth will be slow, with some countries potentially experiencing a recession.

China's economy will face challenges, including a sluggish property market, declining household wealth, and weak exports. In contrast, India is expected to be the fastest-growing large economy, while the Middle East will experience modest growth driven by low oil prices. South Africa's growth will be limited due to a troubled energy market, and Australia will see moderate growth as China's weakness affects commodity demand.

The retail industry is also facing challenges, including suppressed consumer spending, macroeconomic uncertainty, and rapid changes in consumer behavior. Despite these cha

## Appendix

In [170]:
# Unused combined prompts

combine_prompt = """
You will be given a series of texts below all from a single long article, delimited by triple backquotes.
You are writing a newsletter section condensing out the important points in the article, each supported by a paragraph of around 200 words
The target audience is company leadership, so be concise and rigorous.
Final summary should be at around 500 words long and not only consist of bullet points.

```{text}```

SUMMARY:
"""

combine_prompt = """
You will be given a series of texts below within the triple backquotes, all summaries of sections from the SAME article.
You are a Walmart market researcher writing a newsletter section condensing out the important points in the article. Give sufficient support to these points.
The target audience is executive leadership, final summary should be at around 500 words long and NOT USE bullet points.

```{text}```

NEWSLETTER ENTRY:
"""
